In [ ]:
# SVT Method
def svt_solve(A, Omega, tau=None, delta=None, epslion=1e-2, max_iterations=1000):
    #  矩阵初始化，生成一个和矩阵A形状一样的0矩阵
    Y = np.zeros_like(A)

    if not tau:
        tau = 5 * np.sum(A.shape) / 2
    if not delta:
        #  确定步长初始值
        delta = 1.2 * np.prod(A.shape) / np.sum(Omega)
    for _ in range(max_iterations):
        #  对矩阵Y进行奇异值分解
        U, S, V = np.linalg.svd(Y, full_matrices=False)
        #  soft-thresholding operator
        #print(type(S))
        #print(type(tau))
        #print(tau)
        S = np.maximum(S - tau, 0)
        #  singular value shrinkage
        X = np.linalg.multi_dot([U, np.diag(S), V])
        #  Y的迭代
        Y += delta * Omega * (A-X)
        #  误差计算
        rel_recon_error = np.linalg.norm(Omega * (X-A)) / np.linalg.norm(Omega*A)
        if rel_recon_error < epslion:
            break
    return X

# KNN Method
from sklearn.impute import KNNImputer
# Soft Impute
from fancyimpute import SoftImpute

In [ ]:
import numpy as np
from pdb import lasti2lineno
from pandas.io.parsers import readers
import pandas as pd
import csv

original_name = 'RPunch1415_Original.csv'
yhat1_name = 'asymPunch1415_c4.csv'
yhat2_name = 'asymPunch1415_c5.csv'
yhat3_name = 'asymPunch1415_c6.csv'

with open(original_name) as f:
  reader1 = csv.reader(f)
  lst1 = list(reader1)
  original = np.array(lst1)
  #print(original)

with open(yhat1_name) as f:
  reader21 = csv.reader(f)
  lst21 = list(reader21)
  yhat1 = np.array(lst21)
  #print(yhat1)

with open(yhat2_name) as f:
  reader22 = csv.reader(f)
  lst22 = list(reader22)
  yhat2 = np.array(lst22)
  #print(yhat2)

with open(yhat3_name) as f:
  reader23 = csv.reader(f)
  lst23 = list(reader23)
  yhat3 = np.array(lst23)
  #print(yhat3)

original = np.delete(original,0,0)
original = np.delete(original,0,1)
#print(original)

yhat1 = np.delete(yhat1,0,0)
yhat1 = np.delete(yhat1,0,1)
yhat2 = np.delete(yhat2,0,0)
yhat2 = np.delete(yhat2,0,1)
yhat3 = np.delete(yhat3,0,0)
yhat3 = np.delete(yhat3,0,1)
#print(yhat1)
#print(yhat2)
#print(yhat3)

raw_data1 = original.copy()
raw_data2 = original.copy()
raw_data3 = original.copy()

for y in range(yhat1.shape[1]):
  for x in range(yhat1.shape[0]):
    if(yhat1[x,y] == "0"):
      raw_data1[x,y] = 0


for y in range(yhat2.shape[1]):
  for x in range(yhat2.shape[0]):
    if(yhat2[x,y] == "0"):
      raw_data2[x,y] = 0


for y in range(yhat3.shape[1]):
  for x in range(yhat3.shape[0]):
    if(yhat3[x,y] == "0"):
      raw_data3[x,y] = 0

raw_data1 = raw_data1.astype(float)
raw_data2 = raw_data2.astype(float)
raw_data3 = raw_data3.astype(float)

# generating Omega :0 denotes None 1 denotes true
shape1 = raw_data1.shape
Omega1 = np.ones(shape1)
for i in range(0, shape1[0]):
    for j in range(0, shape1[1]):
        if yhat1[i,j] == "0":
            Omega1[i, j] = 0

shape2 = raw_data2.shape
Omega2 = np.ones(shape2)
for i in range(0, shape2[0]):
    for j in range(0, shape2[1]):
        if yhat2[i,j] == "0":
            Omega2[i, j] = 0

shape3 = raw_data3.shape
Omega3 = np.ones(shape3)
for i in range(0, shape3[0]):
    for j in range(0, shape3[1]):
        if yhat3[i,j] == "0":
            Omega3[i, j] = 0
#print(Omega1)
#print(Omega2)
#print(Omega3)

raw_data_na1 = raw_data1.copy()
for x in range(yhat1.shape[0]):
  for y in range(yhat1.shape[1]):
    if raw_data1[x,y] == 0:
      raw_data_na1[x,y] = np.nan

raw_data_na2 = raw_data2.copy()
for x in range(yhat2.shape[0]):
  for y in range(yhat2.shape[1]):
    if raw_data2[x,y] == 0:
      raw_data_na2[x,y] = np.nan

raw_data_na3 = raw_data3.copy()
for x in range(yhat3.shape[0]):
  for y in range(yhat3.shape[1]):
    if raw_data3[x,y] == 0:
      raw_data_na3[x,y] = np.nan

In [ ]:
result1_Avg = raw_data1.copy()
result2_Avg = raw_data2.copy()
result3_Avg = raw_data3.copy()

for i in range(original.shape[0]):
  for j in range(original.shape[1]):
    if result1_Avg[i,j] == 0:
      result1_Avg[i,j] = np.mean([num for num in result1_Avg[:,j] if num!= 0])

for i in range(original.shape[0]):
  for j in range(original.shape[1]):
    if result2_Avg[i,j] == 0:
      result2_Avg[i,j] = np.mean([num for num in result2_Avg[:,j] if num!= 0])

for i in range(original.shape[0]):
  for j in range(original.shape[1]):
    if result3_Avg[i,j] == 0:
      result3_Avg[i,j] = np.mean([num for num in result3_Avg[:,j] if num!= 0])

result1_SVT = svt_solve(raw_data1, Omega1)
result2_SVT = svt_solve(raw_data2, Omega2)
result3_SVT = svt_solve(raw_data3, Omega3)

imputer25 = KNNImputer(n_neighbors = 25, weights = "uniform")
result1_KNN25 = imputer25.fit_transform(raw_data_na1)
result2_KNN25 = imputer25.fit_transform(raw_data_na2)
result3_KNN25 = imputer25.fit_transform(raw_data_na3)

result1_Soft = SoftImpute(verbose=False).fit_transform(raw_data_na1)
result2_Soft = SoftImpute(verbose=False).fit_transform(raw_data_na2)
result3_Soft = SoftImpute(verbose=False).fit_transform(raw_data_na3)

In [ ]:
from fancyimpute.knn import KNN
import math

original = original.astype(float)
sqEr1_SVT = 0
sqEr1_KNN25 = 0
sqEr1_Soft = 0
sqEr1_Avg = 0

sqEr2_SVT = 0
sqEr2_KNN25 = 0
sqEr2_Soft = 0
sqEr2_Avg = 0

sqEr3_SVT = 0
sqEr3_KNN25 = 0
sqEr3_Soft = 0
sqEr3_Avg = 0

count1 = 0
count2 = 0
count3 = 0

SVT_error_doc = []
KNN_error_doc = []
Soft_error_doc = []
Avg_error_doc = []

for y in range(raw_data1.shape[1]):
  for x in range(raw_data1.shape[0]):
    if(raw_data1[x,y]== 0):
      count1 = count1 + 1
      sqEr1_SVT = sqEr1_SVT + abs(1 - result1_SVT[x,y]/original[x,y])
      SVT_error_doc.append(1 - result1_SVT[x,y]/original[x,y])

      sqEr1_KNN25 = sqEr1_KNN25 + abs(1 - result1_KNN25[x,y]/original[x,y])
      KNN_error_doc.append(1 - result1_KNN25[x,y]/original[x,y])

      sqEr1_Soft = sqEr1_Soft + abs(1- result1_Soft[x,y]/original[x,y])
      Soft_error_doc.append(1 - result1_Soft[x,y]/original[x,y])

      sqEr1_Avg = sqEr1_Avg + abs(1- result1_Avg[x,y]/original[x,y])
      Avg_error_doc.append(1 - result1_Avg[x,y]/original[x,y])

for y in range(raw_data2.shape[1]):
  for x in range(raw_data2.shape[0]):
    if(raw_data2[x,y] == 0):
      count2 = count2 + 1
      sqEr2_SVT = sqEr2_SVT + abs(1 - result2_SVT[x,y]/original[x,y])
      SVT_error_doc.append(1 - result2_SVT[x,y]/original[x,y])

      sqEr2_KNN25 = sqEr2_KNN25 + abs(1- result2_KNN25[x,y]/original[x,y])
      KNN_error_doc.append(1 - result2_KNN25[x,y]/original[x,y])

      sqEr2_Soft = sqEr2_Soft + abs(1 - result2_Soft[x,y]/original[x,y])
      Soft_error_doc.append(1 - result2_Soft[x,y]/original[x,y])

      sqEr2_Avg = sqEr2_Avg + abs(1 - result2_Avg[x,y]/original[x,y])
      Avg_error_doc.append(1 - result2_Avg[x,y]/original[x,y])

for y in range(raw_data3.shape[1]): # change according to size
  for x in range(raw_data3.shape[0]): # change according to size
    if(raw_data3[x,y]== 0):
      count3 = count3 + 1
      sqEr3_SVT = sqEr3_SVT + abs(1 - result3_SVT[x,y]/original[x,y])
      SVT_error_doc.append(1 - result3_SVT[x,y]/original[x,y])

      sqEr3_KNN25 = sqEr3_KNN25 + abs(1 - result3_KNN25[x,y]/original[x,y])
      KNN_error_doc.append(1 - result3_KNN25[x,y]/original[x,y])

      sqEr3_Soft = sqEr3_Soft + abs(1 - result3_Soft[x,y]/original[x,y])
      Soft_error_doc.append(1 - result3_Soft[x,y]/original[x,y])

      sqEr3_Avg = sqEr3_Avg + abs(1 - result3_Avg[x,y]/original[x,y])
      Avg_error_doc.append(1 - result3_Avg[x,y]/original[x,y])

In [ ]:
print("Count")
print(count1)
print(count2)
print(count3)

print("SVT")
print(sqEr1_SVT)
print(sqEr2_SVT)
print(sqEr3_SVT)

print("KNN-25")
print(sqEr1_KNN25)
print(sqEr2_KNN25)
print(sqEr3_KNN25)

print("Soft")
print(sqEr1_Soft)
print(sqEr2_Soft)
print(sqEr3_Soft)


print("Avg")
print(sqEr1_Avg)
print(sqEr2_Avg)
print(sqEr3_Avg)

Count
128
113
574
SVT
12.400504272029947
26.246211086145824
132.10097125854887
KNN-25
12.413361288837983
22.667356634117862
69.83823564882707
Soft
13.048189373995722
25.699314417913754
135.25340421246352
Avg
16.248684441486247
24.94286380141626
83.32271849600784


In [ ]:
from google.colab import files

# Specify the file name
file1_name = "SVT_1415_asym2_doc.txt"
file2_name = "KNN_1415_asym2_doc.txt"
file3_name = "Soft_1415_asym2_doc.txt"
file4_name = "Avg_1415_asym2_doc.txt"

SVT_error_doc = [str(item) for item in SVT_error_doc]
KNN_error_doc = [str(item) for item in KNN_error_doc]
Soft_error_doc = [str(item) for item in Soft_error_doc]
Avg_error_doc = [str(item) for item in Avg_error_doc]

# Create and open the file in write mode
with open(file1_name, 'w') as file:
  for item in SVT_error_doc:
        file.write(item + '\n')

with open(file2_name, 'w') as file:
  for item in KNN_error_doc:
        file.write(item + '\n')

with open(file3_name, 'w') as file:
  for item in Soft_error_doc:
        file.write(item + '\n')

with open(file4_name, 'w') as file:
  for item in Avg_error_doc:
        file.write(item + '\n')

# Download the file
files.download(file1_name)
files.download(file2_name)
files.download(file3_name)
files.download(file4_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Soft Impute Install
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.6 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29882 sha256=582af9410fb25fd9f3054f12ec331b999c63bb14802f9f8a2fedbb51b13e39a9
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=032f3e2fcd3356fab49b9c9207b8d384180a7beb3a543938f10b96c6d2f281b1
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [ ]:
np.set_printoptions(threshold=np.inf)
print(result3_SVT)

[[3.42945047 5.62929891 4.30316014 6.17008852 3.11083531]
 [4.63408877 5.90560689 6.11661729 5.75211788 2.92695858]
 [4.27001825 6.28782389 5.9738086  5.96147043 2.94281809]
 [4.21025342 6.41040105 5.52075497 5.73588097 2.70807932]
 [4.20780606 5.70533378 6.29588482 5.41259907 3.10096514]
 [3.95050957 5.67239584 5.72158725 5.42434829 2.99745867]
 [4.42158908 5.82390473 6.21114806 6.02287085 3.50830727]
 [4.40250976 6.19228924 5.70816049 5.84278165 3.08596926]
 [4.50422911 6.2548443  5.57469945 5.61052169 3.01713803]
 [3.9730748  5.73481933 5.54271574 6.11461292 3.33008916]
 [4.5186539  6.28345748 6.46168242 5.956748   3.38405465]
 [3.93061833 5.71983713 6.00106391 5.7239308  3.02008427]
 [3.96131539 5.68992628 6.075383   5.95700621 2.90457789]
 [4.33516353 6.2656263  6.54204125 6.49858218 3.24070646]
 [4.07478937 6.17559069 6.46036878 6.30722739 3.39488584]
 [4.30895492 6.47086296 6.03047432 5.19327102 3.37012265]
 [4.09801145 5.58081437 6.24854857 6.33292202 2.90599273]
 [4.55657563 6